# Summarize unstructured customer reviews with Snowflake Cortex
Understanding customer feedback is critical for businesses, but analyzing large volumes of unstructured text can be challenging. In this notebook, you'll use Snowflake Cortex to **summarize** unstructured customer feedback.

By the end, you'll see how Snowflake can help extract insights from unstructured data quickly and effectively.

### Context
*Tasty Bytes* is a global food truck network operating in 15 countries with a fleet of 450 trucks. They collect customer reviews to gain insights into their food truck operations, with feedback coming from multiple sources.

In this notebook, we will analyze these collated customer reviews using Snowflake Cortex to understand what our international customers are saying.

##

## Import sample data

In this next SQL query, we will populate sample data that is used in this and other templates.

In [ ]:
USE ROLE SNOWFLAKE_LEARNING_ROLE;

-- use the existing database, schema and warehouse
USE DATABASE SNOWFLAKE_LEARNING_DB;

SET schema_name = CONCAT(current_user(), '_SUMMARIZE_UNSTRUCTURED_CUSTOMER_REVIEWS');
USE SCHEMA IDENTIFIER($schema_name);

  /*--
  • file format and stage creation
  --*/

  CREATE OR REPLACE FILE FORMAT csv_ff 
    TYPE = 'csv';

  CREATE OR REPLACE STAGE s3load
    COMMENT = 'Quickstarts S3 Stage Connection'
    URL = 's3://sfquickstarts/tastybytes-voc/'
    FILE_FORMAT = csv_ff;


  /*--
  • raw zone table build 
  --*/

  -- truck_reviews table
  CREATE OR REPLACE TABLE truck_reviews
  (
      order_id NUMBER(38,0),
      language VARCHAR(16777216),
      source VARCHAR(16777216),
      review VARCHAR(16777216),
      review_id NUMBER(18,0)
  );
  
  /*--
  • raw zone table load 
  --*/
  
  -- truck_reviews table load
  COPY INTO truck_reviews
  FROM @s3load/raw_support/truck_reviews/;

-- setup completion note
SELECT 'Setup is complete' AS note;


## Import python packages

Snowflake Notebooks include Streamlit and the third-party packages listed in the Snowflake Anaconda channel. 

Now that the necessary packages are installed, we will import the installed packages into the notebook.

In [ ]:
# Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F

# Cortex Functions
import snowflake.cortex  as cortex

session = get_active_session()

## Overview of `TRUCK_REVIEWS` table

In [ ]:
SELECT * FROM TRUCK_REVIEWS LIMIT 15;

## Summarize reviews for each food truck
To quickly extract key insights, we can use Snowflake Cortex's **summarize** function. This will generate concise summaries of the aggregated customer reviews for each truck brand. We can accomplish this in both Python and SQL.

### Summarize an entire column of a Python dataframe

In the following Python cell, we use **Cortex's `cortex.summarize()` function** to generate a summary of all reviews for each brand.

In [ ]:
reviews_df = session.table('truck_reviews')

summarized_reviews_df = reviews_df.withColumn('SUMMARY', cortex.summarize(F.col('REVIEW')))
                                   
summarized_reviews_df.select('SUMMARY').show(15, max_width = 125)

### Summarize an entire column of text in SQL
In the following SQL query, we use **Cortex's `snowflake.cortex.summarize()`** to generate a summary of each review for each brand.

In [ ]:
SELECT 
  REVIEW,
  SNOWFLAKE.CORTEX.SUMMARIZE(REVIEW) AS REVIEW_SUMMARY
FROM TRUCK_REVIEWS
LIMIT 15;


# Conclusion
In this template, we've demonstrated how to summarize unstructured customer reviews using Snowflake Cortex. 

This approach allows businesses to quickly gain actionable insights from customer reviews, enabling data-driven decision-making. 

To learn more about Cortex Summarize and other Cortex LLM Functions, check out the [Cortex Summarize](https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex) and [Cortex LLM Functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions) documentation.